In [1]:
import joblib
import pandas as pd
from RainPredictionMachine.Trainer import *
from google.cloud import storage
import os
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from RainPredictionMachine.data import CleanDataRpm


2022-03-28 18:02:18.501342: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 18:02:18.501405: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def download_model(estacao,bucket='rain-prediction-machine', rm=True):
    client = storage.Client().bucket(bucket)

    storage_location = f'models/{estacao}.joblib'
    blob = client.blob(storage_location)
    blob.download_to_filename(f'{estacao}.joblib')
    print("=> estacao downloaded from storage")
    model = joblib.load(f'{estacao}.joblib')
    if rm:
        os.remove(f'{estacao}.joblib')
    return model

In [3]:
model = download_model('BAURU')

=> estacao downloaded from storage


2022-03-28 18:02:21.357260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-28 18:02:21.357313: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-28 18:02:21.357330: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-5ODF5UT): /proc/driver/nvidia/version does not exist
2022-03-28 18:02:21.357585: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
cleaner = CleanDataRpm()
df = cleaner.clean_data(2, gcp=False)

X_train, y_train, X_test, y_test = split_train_test(df,6000,72)

In [5]:
X_train = X_train[:,:,:17]
X_test = X_test[:,:,:17]

In [6]:
X_train.shape

(6000, 48, 17)

In [7]:
X_test.shape

(6000, 48, 17)

In [8]:
# 6000 sequencias para as proximas 24 horas
model.evaluate(X_test,y_test)

188/188 [==============================] - 9s 42ms/step - loss: 1.1557 - mae: 0.1466 - mse: 1.1557 - mape: 10264322.0000


[1.1557224988937378, 0.14656078815460205, 1.1557224988937378, 10264322.0]

In [9]:
# Avaliar nossa previsao
y_pred = model.predict(X_test)
y_pred

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.26333618, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [10]:
# Transformar o y_test e y_pred em DataFrame
df_y_pred = pd.DataFrame(y_pred).T
df_y_test = pd.DataFrame(y_test).T

In [11]:
def classe_chuva(precipitacao):
    mm=precipitacao
    if np.isnan(mm):
        chuva = "NaN"
    if mm == 0:
        chuva = 0 #'nao chove'
    elif mm >0 and mm <=5.0:
        chuva = 1 #'fraca'
    elif mm >5.0 and mm<=25.0:
        chuva = 2 #'moderada'
    else:
        chuva = 3 #'forte'
    return chuva

In [12]:
# Baseado na nossa classe_chuva, definimos os elemento de acordo com a intensidade da precipitacao
df_y_test_categorical = df_y_test.applymap(classe_chuva)
df_y_test_categorical.T.value_counts()

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0     4203
                                                                                  1       32
1  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0       27
0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   1   1       25
                                                                          1   1   0       21
                                                                                        ... 
   2  1  1  1  0  1  1  1  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0        1
      2  1  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0        1
            1  1  1  1  0  1  1   1   1   0   0   0   0   0   0   2   2   1   0   0        1
   0  0  0  0  0  0  0  0  2  1   1   0   1   0   0   1   1   0   1   1   1   

In [13]:
df_y_pred_categorical = df_y_pred.applymap(classe_chuva)
df_y_pred_categorical.T.value_counts()

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0     5206
                     1  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0       68
         1  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0       52
         0  0  0  0  0  0  0  0   0   0   0   1   0   0   0   0   0   0   0   0   0       41
   1  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0       31
                                                                                        ... 
      1  0  0  0  1  0  0  0  0   0   0   0   1   0   0   0   0   0   1   2   2   2        1
                           1  0   0   0   1   0   1   0   0   0   0   1   1   1   2        1
                        1  1  0   0   1   1   1   1   0   0   0   0   1   1   1   2        1
            1  1  1  0  0  0  0   0   0   1   1   0   0   0   0   1   2   2   

In [14]:
# Igualando para saber o quanto acertamos 
(df_y_test_categorical == df_y_pred_categorical).sum()

0       24
1       21
2       24
3       23
4       18
        ..
5995    22
5996    20
5997    22
5998    24
5999    24
Length: 6000, dtype: int64

In [15]:
(df_y_test_categorical == df_y_pred_categorical).sum().sum()

132584

In [16]:
(df_y_test_categorical == df_y_pred_categorical).sum().sum()/(6000*24) #accuracy  (n_sequences, length)

0.9207222222222222

In [17]:
(df_y_test_categorical == df_y_pred_categorical).sum(axis=1)/6000

0     0.923167
1     0.922667
2     0.923500
3     0.915667
4     0.920000
5     0.928333
6     0.927000
7     0.911500
8     0.926833
9     0.927833
10    0.931667
11    0.923167
12    0.925833
13    0.925167
14    0.912000
15    0.922667
16    0.917833
17    0.915833
18    0.922667
19    0.919333
20    0.907667
21    0.917833
22    0.914333
23    0.914833
dtype: float64

In [18]:
df_y_test_categorical.shape

(24, 6000)

In [20]:
df_y_pred_categorical.shape

(24, 6000)

In [21]:
numpy_y_pred = df_y_pred_categorical.to_numpy()[0,:]
numpy_y_test = df_y_test_categorical.to_numpy()[0,:]

In [22]:
classification_report(numpy_y_test, numpy_y_pred, output_dict=True)

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicte

{'0': {'precision': 0.9341904923024869,
  'recall': 0.9881889763779528,
  'f1-score': 0.9604313418558135,
  'support': 5588},
 '1': {'precision': 0.14457831325301204,
  'recall': 0.031496062992125984,
  'f1-score': 0.05172413793103449,
  'support': 381},
 '2': {'precision': 0.8333333333333334,
  'recall': 0.16666666666666666,
  'f1-score': 0.2777777777777778,
  'support': 30},
 '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1},
 'accuracy': 0.9231666666666667,
 'macro avg': {'precision': 0.4780255347222081,
  'recall': 0.29658792650918636,
  'f1-score': 0.3224833143911564,
  'support': 6000},
 'weighted avg': {'precision': 0.8833901347226156,
  'recall': 0.9231666666666667,
  'f1-score': 0.8991550946958904,
  'support': 6000}}

In [23]:
classification_report(numpy_y_test, numpy_y_pred, output_dict=True)['0']

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicte

{'precision': 0.9341904923024869,
 'recall': 0.9881889763779528,
 'f1-score': 0.9604313418558135,
 'support': 5588}

In [24]:
classification_reports = []

for i in range(0,24):
    numpy_y_pred = df_y_test_categorical.to_numpy()[i,:]
    numpy_y_test = df_y_pred_categorical.to_numpy()[i,:]
    dict_classificacao = classification_report(numpy_y_test, numpy_y_pred, output_dict=True)
    dicionario2 = {}
    for key in list(dict_classificacao.keys())[0:5]:
        for key2 in list(dict_classificacao[key].keys()):
            dicionario2[f'{key2}, {key}'] = dict_classificacao[key][key2]
    classification_reports.append(dicionario2)
    
for i in range(0,24):
    numpy_y_pred = df_y_test_categorical.to_numpy()[i,:]
    numpy_y_test = df_y_pred_categorical.to_numpy()[i,:]
    dict_classificacao = classification_report(numpy_y_test, numpy_y_pred, output_dict=True)
    dicionario2 = {}
    for key in list(dict_classificacao.keys())[0:4]:
#         dicionario2 = dict_classificacao[key]
        for key2 in list(dict_classificacao[key].keys()):
            dicionario2[f'{key2}, {key}'] = dict_classificacao[key][key2]
#             dicionario2.pop(key2)
    classification_reports.append(dicionario2)

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_div

AttributeError: 'float' object has no attribute 'keys'